In [1]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2020-05-15 07:04:28--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-05-15 07:04:28--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-05-15 07:04:29--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [0]:
import numpy as np

In [0]:
from sklearn.preprocessing import LabelBinarizer

In [0]:
import pandas as pd

In [0]:
from sklearn.metrics import accuracy_score

In [0]:
!unzip -q glove.6B.zip

In [0]:
w2v = dict()

In [0]:
words = set()

In [0]:
w2i = dict()
i2w = dict()

In [0]:
with open('glove.6B.50d.txt') as f:
  for i in f:
    i = i.strip().split()
    words.add(i[0])
    w2v[i[0]] = np.array(i[1:],dtype = np.float64)
  for i,x in enumerate(sorted(words)):
    w2i[x] = i
    i2w[i] = x

In [0]:
def sentence_avg(line,w2v):
  words = line.lower().split()
  total = 0
  for w in words:
    total+=w2v[w]
  avg = total/len(words)
  return avg

In [0]:
def softmax(x):
  e_x = np.exp(x - np.max(x))
  return e_x / e_x.sum()

In [0]:
def preds(x,y,W,b,w2v):
  n = y.shape[0]
  pred = np.zeros((n,1))
  for i in range(n):
    words = x[i].lower().split()
    avg = np.zeros((50,))
    for w in words:
      avg+=w2v[w]
    avg = avg/len(words)
    z = b + np.dot(W,avg)
    a = softmax(z)
    pred[i] = np.argmax(a)
  return pred

In [0]:
def model(x,y,w2v,epochs = 100,lr = 0.01):
  ny = 5
  nh = 50
  n = y.shape[0]
  W = np.random.randn(ny,nh)/np.sqrt(nh)
  b = np.zeros(ny,)
  lb = LabelBinarizer()
  y = lb.fit_transform(y)
  for e in range(epochs):
    for i in range(n):
      avg = sentence_avg(x[i],w2v)
      z = b + np.dot(W,avg)
      a = softmax(z)
      loss = -np.sum(y[i]*np.log(a))
      dz = a - y[i]
      dw = np.dot(dz.reshape(ny,1), avg.reshape(1, nh))
      db = dz 
      W-=lr*dw
      b-=lr*db
    if e%100==0:
      print(e,':',loss)
      pred = preds(x,y,W,b,w2v)
      print(accuracy_score(lb.inverse_transform(y),pred))
  return pred,W,b

In [0]:
train = pd.read_csv('train_emoji.csv',header = None)

In [0]:
x = np.array(train[0])
y = np.array(train[1])

In [51]:
pred,W,b = model(x,y,w2v,epochs = 600)

0 : 1.7195022195617633
0.4696969696969697
100 : 0.06452128349393395
0.9242424242424242
200 : 0.03812979450028687
0.9545454545454546
300 : 0.0306962248856829
0.9696969696969697
400 : 0.02677093852429714
0.9772727272727273
500 : 0.024075328344333652
0.9772727272727273


In [0]:
test = pd.read_csv('tesss.csv',header = None)

In [0]:
testx = test[0]
testy = test[1]

In [0]:
testx = np.array(testx)

In [0]:
testpred = preds(testx,testy,W,b,w2v)

In [62]:
print(accuracy_score(testy,testpred))

0.875


In [0]:
emoji_dict = {0:':heart:',1:':baseball:',2:':smile:',3:':disappointed:',4:':fork_and_knife:'}

In [66]:
!pip install emoji

     |████████████████████████████████| 51kB 1.6MB/s 
  Created wheel for emoji: filename=emoji-0.5.4-cp36-none-any.whl size=42176 sha256=8f030843e4d9dc3740228d9b183db910bf94e3882d3b9eb17cc4a096209634c2
  Stored in directory: /root/.cache/pip/wheels/2a/a9/0a/4f8e8cce8074232aba240caca3fade315bb49fac68808d1a9c
Successfully built emoji


In [0]:
import emoji

In [70]:
emoji.emojize(emoji_dict[1],use_aliases=True)

'⚾'

In [0]:
demo = pd.DataFrame()
demo['Text'] = testx
detmo['Original'] = [emoji.emojize(emoji_dict[x],use_aliases=True) for x in testy]

In [0]:
demo['Predicted'] = [emoji.emojize(emoji_dict[x],use_aliases=True) for x in testpred.ravel()]

In [101]:
demo

,Text,Original,Predicted
0,I want to eat\t,🍴,🍴
1,he did not answer\t,😞,😞
2,he got a very nice raise\t,😄,😄
3,she got me a nice present\t,😄,😄
4,ha ha ha it was so funny\t,😄,😄
5,he is a good friend\t,😄,😄
6,I am upset\t,😞,😞
7,We had such a lovely dinner tonight\t,😄,😄
8,where is the food\t,🍴,🍴
9,Stop making this joke ha ha ha\t,😄,😄
